<a href="https://colab.research.google.com/github/Prasad-py/Twitter_Sentiment_Detection/blob/main/Twitter_sentiment_analysis_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd 
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
import tensorflow_hub as hub
import tensorflow_text as text
from sklearn.utils import shuffle

In [2]:
!pip install tensorflow_text
!pip install sentencepiece

In [3]:
tf.version

<module 'tensorflow._api.v2.version' from '/usr/local/lib/python3.7/dist-packages/tensorflow/_api/v2/version/__init__.py'>

In [4]:
# from transformers import BertTokenizer
# tz = BertTokenizer.from_pretrained("bert-base-cased")

# sent = "Let's learn deep learning!"

# # Encode the sentence
# encoded = tz.encode_plus(
#     text=sent,  # the sentence to be encoded
#     add_special_tokens=True,  # Add [CLS] and [SEP]
#     max_length = 64,  # maximum length of a sentence
#     pad_to_max_length=True, # Add [PAD]s
#     truncation=True,
#     return_attention_mask = True,  # Generate the attention mask
# #     return_tensors = 'pt',  # ask the function to return PyTorch tensors
# )

# # Get the input IDs and attention mask in tensor format
# input_ids = encoded['input_ids']
# attn_mask = encoded['attention_mask']

# print(input_ids)
# print(attn_mask)

In [5]:
!wget  https://raw.githubusercontent.com/tensorflow/models/master/official/nlp/bert/tokenization.py

--2022-01-18 08:01:05--  https://raw.githubusercontent.com/tensorflow/models/master/official/nlp/bert/tokenization.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16591 (16K) [text/plain]
Saving to: ‘tokenization.py.8’

tokenization.py.8   100%[===================>]  16.20K  --.-KB/s    in 0s      

2022-01-18 08:01:05 (39.2 MB/s) - ‘tokenization.py.8’ saved [16591/16591]



In [6]:
import tokenization

In [7]:
def bert_encode(texts, tokenizer, max_len=512):
    all_tokens = []
    all_masks = []
    all_segments = []
    
    for text in texts:
        text = tokenizer.tokenize(text)
            
        text = text[:max_len-2]
        input_sequence = ["[CLS]"] + text + ["[SEP]"]
        pad_len = max_len - len(input_sequence)
        
        tokens = tokenizer.convert_tokens_to_ids(input_sequence)
        tokens += [0] * pad_len
        pad_masks = [1] * len(input_sequence) + [0] * pad_len
        segment_ids = [0] * max_len
        
        all_tokens.append(tokens)
        all_masks.append(pad_masks)
        all_segments.append(segment_ids)
    
    return np.array(all_tokens), np.array(all_masks), np.array(all_segments)

In [8]:
def build_model(bert_layer, max_len=512):
    input_word_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    input_mask = Input(shape=(max_len,), dtype=tf.int32, name="input_mask")
    segment_ids = Input(shape=(max_len,), dtype=tf.int32, name="segment_ids")

    _, sequence_output = bert_layer([input_word_ids, input_mask, segment_ids])
    clf_output = sequence_output[:, 0, :]
    out = Dense(1, activation='sigmoid')(clf_output)
    
    model = Model(inputs=[input_word_ids, input_mask, segment_ids], outputs=out)
    model.compile(Adam(lr=2e-6), loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

In [9]:
module_url = "https://tfhub.dev/tensorflow/bert_en_uncased_L-24_H-1024_A-16/1"
bert_layer = hub.KerasLayer(module_url, trainable=True)

In [10]:
vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = tokenization.FullTokenizer(vocab_file, do_lower_case)

In [11]:
train_data = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/training_data (2).csv",  encoding='latin-1')
# test_data = pd.read_csv(r"/content/drive/MyDrive/Colab Notebooks/testdata.manual.2009.06.14.csv")

In [12]:
train_data.sample()

,Sentiment,id,date,query,name,text
1013559,4,1881329375,Fri May 22 04:20:22 PDT 2009,NO_QUERY,jessicalace,@mryrose thanks!


In [13]:
train_data = train_data[['Sentiment','text' ]]
# test_data = test_data[['Sentiment','text']]
train_data['senti']=train_data['Sentiment'].apply(lambda x: 1 if x==4 else 0)
# test_data['senti']=test_data['Sentiment'].apply(lambda x: 1 if x==4 else 0)
train_data

,Sentiment,text,senti
0,0,is upset that he can't update his Facebook by ...,0
1,0,@Kenichan I dived many times for the ball. Man...,0
2,0,my whole body feels itchy and like its on fire,0
3,0,"@nationwideclass no, it's not behaving at all....",0
4,0,@Kwesidei not the whole crew,0
...,...,...,...
1048570,4,My GrandMa is making Dinenr with my Mum,1
1048571,4,Mid-morning snack time... A bowl of cheese noo...,1
1048572,4,@ShaDeLa same here say it like from the Termi...,1
1048573,4,@DestinyHope92 im great thaanks wbuu?,1


In [14]:
train_labels = train_data['senti'].tolist()
train_text = train_data['text'].tolist()
# test_labels = test_data['Sentiment'].tolist()
# test_text = test_data['text'].tolist()
train_labels,train_text = shuffle(train_labels,train_text)
train_text = train_text[1:3500]
train_labels = train_labels[1:3500]
train_text
del train_data
# del test_data

In [15]:
train_input = bert_encode(train_text, tokenizer, max_len=160)
# test_input = bert_encode(test_text, tokenizer, max_len=160)
train_input[0]

array([[  101,  2975, 25906, ...,     0,     0,     0],
       [  101,  2025,  2130, ...,     0,     0,     0],
       [  101,  1030,  2019, ...,     0,     0,     0],
       ...,
       [  101,  1045,  4299, ...,     0,     0,     0],
       [  101,  1030,  1050, ...,     0,     0,     0],
       [  101,  2074,  3062, ...,     0,     0,     0]])

In [16]:
model = build_model(bert_layer, max_len=160)
model.summary()
# model.add(Dropout(0.2))


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_word_ids (InputLayer)    [(None, 160)]        0           []                               
                                                                                                  
 input_mask (InputLayer)        [(None, 160)]        0           []                               
                                                                                                  
 segment_ids (InputLayer)       [(None, 160)]        0           []                               
                                                                                                  
 keras_layer (KerasLayer)       [(None, 1024),       335141889   ['input_word_ids[0][0]',         
                                 (None, 160, 1024)]               'input_mask[0][0]',         

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [17]:
# tf.convert_to_tensor(
#     train_text, dtype=None, dtype_hint=None, name=None
# )

# train_labels_new = np.asarray(train_labels)
# train_text_new = np.asarray(train_input)
# train_text_neww = np.resize(train_text_new , (3, 4999, 160))
# train_text_neww[0]
train_text_new = np.asarray(train_input)
train_text_new_1 = np.resize(train_text_new , (4999, 3, 160))
train_input_new = []
for (a,b,c) in train_text_new_1:
    train_input_new.append([tf.convert_to_tensor(np.asarray(a)), tf.convert_to_tensor(np.asarray(b)), tf.convert_to_tensor(np.asarray(c))])
train_input_new = np.asarray(train_input_new)
# train

In [18]:
# train_input_new[0]  

In [19]:
train_history = model.fit(
    [np.asarray(train_input[0]),np.asarray(train_input[1]),np.asarray(train_input[2])], np.asarray(train_labels),
    validation_split=0.3,
    epochs=5,
    batch_size=16,
    verbose=1
)

Epoch 1/5
154/154 [==============================] - 314s 2s/step - loss: 0.5107 - accuracy: 0.7648 - val_loss: 0.4387 - val_accuracy: 0.7990
Epoch 2/5
154/154 [==============================] - 282s 2s/step - loss: 0.3494 - accuracy: 0.8469 - val_loss: 0.3837 - val_accuracy: 0.8295
Epoch 3/5
154/154 [==============================] - 283s 2s/step - loss: 0.2402 - accuracy: 0.9085 - val_loss: 0.3693 - val_accuracy: 0.8381
Epoch 4/5
154/154 [==============================] - 282s 2s/step - loss: 0.1474 - accuracy: 0.9518 - val_loss: 0.3762 - val_accuracy: 0.8448
Epoch 5/5
154/154 [==============================] - 282s 2s/step - loss: 0.0773 - accuracy: 0.9849 - val_loss: 0.4090 - val_accuracy: 0.8571


In [20]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [21]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_word_ids (InputLayer)    [(None, 160)]        0           []                               
                                                                                                  
 input_mask (InputLayer)        [(None, 160)]        0           []                               
                                                                                                  
 segment_ids (InputLayer)       [(None, 160)]        0           []                               
                                                                                                  
 keras_layer (KerasLayer)       [(None, 1024),       335141889   ['input_word_ids[0][0]',         
                                 (None, 160, 1024)]               'input_mask[0][0]',         